<a href="https://colab.research.google.com/github/ShreyJais/LLM/blob/main/Shrey_558_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install tensorflow
#!pip install pandas
#!pip install plotly
#!pip install matplotlib
#!pip install tqdm
#!pip install transformers
#!pip install tf-keras
#!pip install scikit-learn

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping


import transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [3]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.4 MB/s eta 0:00:00
   ━

In [1]:
import pandas as pd
#dataset : https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus
data = pd.read_csv("/content/ner_dataset.csv",encoding = 'ISO-8859-1')
data.head(55)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [2]:
data.isnull().sum()

,0
Sentence #,276498
Word,3
POS,0
Tag,1


In [3]:
#data where word is missing
data[data["Word"].isnull()]

,Sentence #,Word,POS,Tag
197658,Sentence: 9047,NaN,NN,O
256026,Sentence: 11709,NaN,NN,O
257069,Sentence: 11759,NaN,NN,O


In [4]:
data[197658:]

,Sentence #,Word,POS,Tag
197658,Sentence: 9047,NaN,NN,O
197659,NaN,of,IN,O
197660,NaN,the,DT,O
197661,NaN,32,CD,O
197662,NaN,freed,VBN,O
...,...,...,...,...
289765,NaN,on,IN,O
289766,NaN,Iran,NNP,B-geo
289767,NaN,to,TO,O
289768,NaN,stop,VB,O


In [5]:
# fill N/A values in the column sentence with previous values
data["Sentence #"] = data["Sentence #"].fillna(method="ffill")

<ipython-input-5-c0897ab6f0f1>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data["Sentence #"] = data["Sentence #"].fillna(method="ffill")


In [6]:
data[197658:]

,Sentence #,Word,POS,Tag
197658,Sentence: 9047,NaN,NN,O
197659,Sentence: 9047,of,IN,O
197660,Sentence: 9047,the,DT,O
197661,Sentence: 9047,32,CD,O
197662,Sentence: 9047,freed,VBN,O
...,...,...,...,...
289765,Sentence: 13272,on,IN,O
289766,Sentence: 13272,Iran,NNP,B-geo
289767,Sentence: 13272,to,TO,O
289768,Sentence: 13272,stop,VB,O


In [7]:
data.isnull().sum()

,0
Sentence #,0
Word,3
POS,0
Tag,1


In [8]:
data = data.dropna()

In [9]:
data.Tag.unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [10]:
data.Tag.unique()
print(f"Number of Tags : {len(data.Tag.unique())}")

Number of Tags : 17


In [11]:
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
289764,Sentence: 13272,calling,VBG,O
289765,Sentence: 13272,on,IN,O
289766,Sentence: 13272,Iran,NNP,B-geo
289767,Sentence: 13272,to,TO,O


In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"])
data

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
...,...,...,...,...
289764,3638,calling,VBG,O
289765,3638,on,IN,O
289766,3638,Iran,NNP,B-geo
289767,3638,to,TO,O


In [14]:
data[:60]

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-geo
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


In [15]:
data.rename(columns={"sentence #":"sentence_idf", "word": "words", "Tag": "labels"}, inplace =True)

In [16]:
data["labels"] = data["labels"].str.upper()

In [17]:
data

,Sentence #,Word,POS,labels
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
...,...,...,...,...
289764,3638,calling,VBG,O
289765,3638,on,IN,O
289766,3638,Iran,NNP,B-GEO
289767,3638,to,TO,O


In [18]:
X = data[["Sentence #", "Word"]]
y = data["labels"]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":X_train["Sentence #"], "words" :X_train["Word"], "labels" :y_train})
test_data = pd.DataFrame({"sentence_id" :X_test["Sentence #"], "words" :X_test ["Word"], "labels":y_test})

In [21]:
train_data

,sentence_id,words,labels
209265,12810,Shortly,O
287101,3494,you,O
50473,4709,the,B-GEO
104443,7441,the,O
8662,6551,the,O
...,...,...,...
119879,8234,a,O
259181,2069,Baghdad,B-GEO
131932,8858,officials,O
146867,9602,said,O


In [22]:
test_data

,sentence_id,words,labels
215919,13154,record,O
273511,2805,on,O
234415,812,were,O
283495,3317,plan,O
107221,7577,people,O
...,...,...,...
20973,12695,late,O
286221,3452,anti-Syrian,O
186048,11621,play,O
207824,12742,audit,O


# Model Training

In [23]:
#!pip install torch

In [24]:
from simpletransformers.ner import NERModel, NERArgs

In [25]:
label = data["labels"].unique().tolist()
label

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [26]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args. train_batch_size = 32
args.eval_batch_size = 32

In [27]:
model = NERModel('bert', 'bert-base-cased', labels=label, args = args, use_cuda = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [28]:
model.train_model(train_data, eval_data = test_data, acc = accuracy_score)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/415 [00:00<?, ?it/s]

(415, 0.261245025586651)

In [29]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/404 [00:00<?, ?it/s]

In [30]:
result

{'eval_loss': 0.1819979142914019,
 'precision': 0.8191139240506329,
 'recall': 0.7489583333333333,
 'f1_score': 0.782466747279323}

In [31]:
prediction, model_output = model.predict(["HI my name is Shrey and I work at Apple company, and live in Bangalore"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
prediction

[[{'HI': 'O'},
  {'my': 'O'},
  {'name': 'O'},
  {'is': 'O'},
  {'Shrey': 'B-PER'},
  {'and': 'O'},
  {'I': 'O'},
  {'work': 'O'},
  {'at': 'O'},
  {'Apple': 'B-ORG'},
  {'company,': 'O'},
  {'and': 'O'},
  {'live': 'O'},
  {'in': 'O'},
  {'Bangalore': 'B-GEO'}]]